In [ ]:
#fill the location and saved file
location = 'C:/put/the/location/here/file.pdf'
saveas = 'rekeningBCA.xlsx'

In [ ]:
#import library
import pdfplumber
import pandas as pd
pdf = pdfplumber.open()

In [ ]:
data = []
for page in pdf.pages:
    text = page.extract_text()
    #for newer BCA statement, change the line
    for line in text.split("\n")[16:]: 
        # print(line)
        data.append(line)

In [ ]:
#change to dataframe
rekening = pd.DataFrame(data, columns=['text'])

In [ ]:
#remove unused row
rekening = rekening[~(rekening['text'].str.contains('SALDO AWAL|MUTASI CR|MUTASI DB|SALDO AKHIR', regex=True))]

In [60]:
#ekstrak tanggal
rekening['tanggal'] = rekening['text'].str.extract(r'^(\d{2}/\d{2})\s')

#Gabung all text
rekening = rekening.groupby((~rekening["tanggal"].isna()).cumsum()).agg({"tanggal":"first","text":" ".join})
rekening = rekening.rename(columns={'tanggal':'tgl'}).reset_index().drop('tanggal', axis=1)

#Ekstrak nilai dari text
rekening['nilai'] = rekening['text'].str.findall(r'(\d{1,3}(?:,\d{3})*(?:\.\d{2}))')
# rekening['nilai'] = rekening['text'].str.findall(r'(\d{1,3}(?:\.\d{3})*,\d{2})')

#buat kolom untuk transaksi, debit, dan saldo
rekening['transaksi'] = rekening['nilai'].str[0]
rekening['debet'] = rekening['text'].str.extract(r'\s(DB)\s')
rekening['saldo'] = rekening['nilai'].str[-1]


In [ ]:
#Buat rekening keterangan
rekening['keterangan'] = rekening['text']
rekening['keterangan'] = rekening.apply(lambda row: row['keterangan'].replace(row['tgl'], '').strip(), axis=1, )
rekening['keterangan'] = rekening.apply(lambda row: row['keterangan'].replace(row['transaksi'], '').strip() if type(row['transaksi']) == str else row['keterangan'], axis=1, )
rekening['keterangan'] = rekening.apply(lambda row: row['keterangan'].replace(row['saldo'], '').strip() if type(row['saldo']) == str else row['keterangan'], axis=1)

#clean unused columns
rekening_bersih = rekening.drop(['text', 'nilai'], axis=1)

#perbaiki kolom saldo
rekening_bersih.loc[rekening_bersih['saldo'] == rekening_bersih['transaksi'], 'saldo'] = 0

#ubah ke float
rekening_bersih['transaksi'] = rekening_bersih['transaksi'].replace(',', '', regex=True).astype('float')
rekening_bersih['saldo'] = rekening_bersih['saldo'].replace(',', '', regex=True).astype('float')

In [ ]:
rekening_bersih.to_excel(saveas)